In [26]:
import requests
import json
import os
from dotenv import load_dotenv
import datetime as dt
load_dotenv()

True

In [58]:
TOKEN = os.getenv('TOKEN')
API_site = "https://api.marketaux.com/v1/news/all"
others_setting = "filter_entities=true&language=en"
yesterday = dt.datetime.now() - dt.timedelta(10)
yesterday = dt.datetime.strftime(yesterday, '%Y-%m-%d')
print(yesterday)

2022-09-29


In [65]:
symbols = ["META", "AMAZ", "AAPL", "NFLX", "GOOGL"]
symbols = ["META"]
stocknews = dict()
for symbol in symbols:
    reqstr = f'{API_site}?symbols={symbol}&{others_setting}&published_before={yesterday}&api_token={TOKEN}'
    data = requests.get(reqstr, timeout=2.50)
    stocknews[symbol] = json.loads(data.text)

In [66]:
print(stocknews['META'])

{'meta': {'found': 1682, 'returned': 3, 'limit': 3, 'page': 1}, 'data': [{'uuid': '1e8e9706-779f-473d-a981-241454b870b5', 'title': 'Move over, TikTok: Social media has ‘a new king in town’ in BeReal', 'description': "Facebook, Instagram, and Snapchat have been sweating the effects of TikTok's ascent, but now it seems that have another upstart rival to worry about.", 'keywords': 'article_normal, Computers/Consumer Electronics, Software, Applications Software, Computer Services, Market Research/Public Relations, Public Relations, Internet Search Engines, Advertising/Marketing/Public Relations, Cloud Computing, Business/Consumer Services, Computing, Data Services, Internet/Online Services, Media/Entertainment, Mobile Applications Software, Social Media Platforms/Tools, Technology, Corporate/Industrial News, Political/General News, Living/Lifestyle, Personal Technology, corporate, industrial news, political, general news, living, lifestyle, personal technology, computers, consumer electron

In [47]:
stocknews['META'] = json.loads(stocknews['META'].text)

In [69]:
for news in stocknews['META']['data']:
    print(news["published_at"])

2022-09-28T23:22:00.000000Z
2022-09-28T14:11:56.000000Z
2022-09-28T13:15:48.000000Z


In [81]:
from datetime import datetime

date_time_str = '2022-09-28T13:15:48.000000Z'

date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%S.000000Z')


print ("The type of the date is now",  type(date_time_obj))
print ("The date is", date_time_obj)

The type of the date is now <class 'datetime.datetime'>
The date is 2022-09-28 13:15:48
